In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/local/openjdk-8"
os.environ["SPARK_HOME"] = "/user_data/spark-3.3.0-bin-hadoop2"

import findspark
findspark.init('spark-3.3.0-bin-hadoop2')

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("ac2")
    .config("spark.sql.warehouse.dir", "hdfs:///user/hive/warehouse")
    .config("spark.sql.catalogImplementation", "hive")
    .getOrCreate()
)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/user_data/spark-3.3.0-bin-hadoop2/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hadoop-2.7.3/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/21 13:10:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
df = spark.read.csv("hdfs://spark-master:9000/datasets/flight/Combined_Flights_2022.csv")

In [13]:
df.show()

+----------+--------------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+-----------+--------------------+---------------+------------------+------------------+--------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+-------------------+---------+-------------+-------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+
|       _c0|                 _c1|   _c2| _c3|      _c4|     _c5|       _c6|    _c7|            _c8|     _c9|   _c10|           _c11|   _c12|          _c13|             _

In [15]:
import requests
from io import StringIO

df = spark.read.csv("hdfs://spark-master:9000/datasets/flight/Combined_Flights_2022.csv", header=True, inferSchema=True)

df.show()

num_linhas = df.count()
print(f"Número de linhas no DataFrame: {num_linhas}")

+-------------------+--------------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+--------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+-------------------+---------+-------------+-------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+
|         FlightDate|             Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|

Número de linhas no DataFrame: 4078318


In [22]:
df_transformado = df.select("Cancelled", "Dest").filter(df["Cancelled"] == 'True')

In [23]:
df_transformado.show()

+---------+----+
|Cancelled|Dest|
+---------+----+
|     true| IAH|
|     true| JAX|
|     true| ECP|
|     true| IAH|
|     true| EWR|
|     true| IAD|
|     true| BWI|
|     true| FLL|
|     true| MCO|
|     true| RSW|
|     true| FLL|
|     true| MCO|
|     true| MCO|
|     true| MIA|
|     true| MCO|
|     true| MCO|
|     true| BDL|
|     true| CHS|
|     true| FLL|
|     true| MCO|
+---------+----+
only showing top 20 rows



In [25]:
# Registrar DataFrame como tabela temporária
df.createOrReplaceTempView("temp_table")

# Executar consulta SQL no DataFrame
resultado_sql = spark.sql("SELECT Cancelled, AVG(DepDelayMinutes) FROM temp_table GROUP BY Cancelled")
resultado_sql.show()

+---------+--------------------+
|Cancelled|avg(DepDelayMinutes)|
+---------+--------------------+
|     true|   68.89432703003337|
|    false|  15.978879560347762|
+---------+--------------------+

